In [1]:
import mediapipe as mp
import numpy as np
import cv2
import os

In [2]:
text_background = (198, 63, 88)  # PURPLE
corner_color = (53, 53, 249)     # RED
text_color = (239, 239, 239)     # WHITE
border_color = (61, 147, 8)      # GREEN

In [3]:
DATA_PATH = os.path.join("Data")

# You, Busy
actions = np.array(["Yes", "WhatAreYouDoing", "TryBeing", "ToMeet", "ThankYou", "TakeCare", "SameAsYou", "Question", "Point", "Nothing", "IHear", "HowAreYou", "Hello", "Good", "Bye", "_BLANK_"])

no_sequences = 40
sequence_length = 30

In [4]:
len(actions)
actions.shape[0]

16

In [5]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [6]:
mp_drawing_styles = mp.solutions.drawing_styles
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)

def mediapipe_detections(frame, model):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame.flags.writeable = False
    results = model.process(frame)
    frame.flags.writeable = True
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    return frame, results


def draw_landmarks(frame, results, color):
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)


def extract_keypoints(results):
    hand_landmarks = np.zeros(63)
    
    if results.multi_hand_landmarks:
        hand_landmarks = np.array(
            [
                [landmark.x, landmark.y, landmark.z]
                for landmark in results.multi_hand_landmarks[0].landmark
            ]
        ).flatten()

    return hand_landmarks

In [12]:
stream_url = "http://192.168.40.200:81/stream"
cap = cv2.VideoCapture(0)

no_frames_counter = 0
no_sequences_counter = 0
index = 6
current_action = actions[index]

while True:
    _, image = cap.read()
    
    image, results = mediapipe_detections(image, hands)
    draw_landmarks(image, results, corner_color)
    right_hand = extract_keypoints(results)
    
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord("s"):
        npy_path = os.path.join(DATA_PATH, current_action, str(no_sequences_counter), f"{no_frames_counter}.npy")
        np.save(npy_path, right_hand)
        no_frames_counter += 1
        if no_frames_counter == sequence_length:
            no_frames_counter = 0
            no_sequences_counter += 1
            if no_sequences_counter == no_sequences:
                break
    
    cv2.putText(image, f"Collecting Frames for '{current_action}'", (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(image, f"Video Num: {no_sequences_counter}", (15, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(image, f"Frame Num: {no_frames_counter}", (15, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.imshow("Image", image)
    
    if key == ord("n"):
        cv2.imwrite(f"{current_action}-image-{no_frames_counter}-{no_sequences_counter}.jpg", image)
    
    if key == ord("q"):
        break
    
cap.release()
cv2.destroyAllWindows()
